# Trabalho Prático 0 - Grupo 15
#### João Esteves - pg46535
#### Sara Queirós - pg47661
## Exercício 1 

1. Criar um comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica  num modo HMAC  que seja seguro contra ataques aos “nounces”.
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves $$\mathtt{cipher\_key}, \mathtt{mac\_key}$$ , para cifra e autenticação, é acordado entre agentes usando o protocolo DH com autenticação dos agentes usando assinaturas DSA.


In [2]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

### Geração de Chaves
* Para efetuar a autenticação do criptograma e dos metadados é necessário criar uma chave com a qual se possa cifrar e decifrar. Utilizando o protocolo DH, geramos os parâmetros necessários para a geração das chaves privadas e públicas que cada um dos intervenientes através de:


In [3]:
parameters = dh.generate_parameters(generator=2, key_size=1024)

#obtenção da chave privada
def get_private_key(self, parameters):
        self.private_key = parameters.generate_private_key()

#obtenção da chave pública
def get_public_key(self):
    self.public_key = self.private_key.public_key()

* Considerando a ideologia do protocolo DH, que é um método que permite que as duas partes concordem conjuntamente com um segredo partilhado, a cada um deles, é enviada a chave pública do outro, a partir da qual, juntamente com a sua chave privada, gera a chave partilhada. Isso é efetuado através da seguinte função, presente no Emitter e Receiver:


In [ ]:
def get_derived_key(self, receiver_public_key):
        key_interm = self.private_key.exchange(receiver_public_key)
        # Derivar chave
        self.shared_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=None,
        ).derive(key_interm) 

**Deste modo é gerada a cipher_key, sendo uma chave simétrica partilhada pelo Emitter e Receiver.**

### Assinaturas DSA
* DSA é um algoritmo que permite assinar as mensagens enviadas, utilizando uma assinatura comum ao Emitter e Receiver, que é cifrada com a chave partilhada por ambos, utilizando o modo HMAC.  
* Posteriormente, o Receiver deve verificar a autenticação da mensagem ao verificar essa assinatura. 


In [ ]:
message = b'This is my signature'

def authentication(self, message):
        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(message)
        return h.finalize()

### Cifrar e Enviar a mensagem
* Para cifrar a mensagem e garantir a sua segurança contra ataques aos nounces, é necessário gerar um nounce aleatório com um Pseudo Random Generator e cifrar a mensagem com esse nounce. A aleatoriedade do nounce segurança adicional à cifra pois não é possível prever o seu valor para cada mensagem.
* Foi utilizada Advanced Encryption Standard utilizando o Galois Encounter Mode, que fornece confidencialidade e integridade ao texto cifrado, permitindo a utilização de um nounce ao cifrar a mensagem. 

In [ ]:
def encode(self, data, ad):
        #codificar o texto 
        nonce = os.urandom(16)
        aesgcm = AESGCM(self.shared_key)
        ad += nonce
        #Cifrar com o nounce e metadados usando AESGCM
        ciphertext = aesgcm.encrypt(nonce, data, ad)
        return nonce, ciphertext

*  A mensagem enviada irá conter a assinatura do Emitter, que permite ao Receiver verificar a autenticidade da origem da mensagem, o nonce (para permitir que a mensagem seja decifrada), assim como os metadados que são enviados, e o texto cifrado.

In [ ]:
def sendMessage(self, mensagem, sigMess):
    #calculos assinatura
    sign = self.authentication(sigMess)
    #gerar um valor aleatorio para os metadados
    ad = os.urandom(12)
    #cifrar o texto e obter o nounce com que foi cifrado.
    nonce, ciphertext = self.encode(mensagem, ad)
    #assinatura + nonce + metadados + texto cifrado
    return sign + nonce + ad + ciphertext

### Receber e Decifrar a mensagem
* Quando o Receiver recebe a mensagem, inicialmente necessita de a dividir nos respetivos elementos de informação para decifrar o texto que ela contém.
* Após isso, verifica se a signature da mensagem que recebeu corresponde à sua signature.Caso ela corresponda, ele decifra a mensagem. Caso contrário não o faz, pois a mensagem não foi enviada pelo Emitter com quem ele espera comunicar.

In [ ]:
def divide_data(data):
        # sign + nonce + ad + ciphertext
        # 0 a 31 - signature : 32 bytes
        # 32 a 47 - nonce : 16 bytes
        # 48 a 59 - ad para derivar
        # a partir do 60 - texto cifrado
        sign = data[:32]
        nonce = data[32:48]
        ad = data[48:60]
        texto_cifrado = data[60:] 
        return sign, nonce, ad, texto_cifrado

#função que lê a mensagem 
def readMessage(self, data, menSig):
    sign, nonce, ad, texto_cifrado = self.divide_data(data)
    #verifica se o autor da mensagem é o esperado
    if(self.verify_Auth(menSig, sign)):
        aad = ad + nonce
        #decifra o texto 
        plaintext = self.decode(texto_cifrado, nonce, aad)
        print("Texto decifrado: ")
        print(plaintext)
    else:
        raise Exception("Autenticidade tem falhas!")

Para verificar a autenticação, gera a sua mac_key, com a chave partilhada e a mesma mensagem do Emitter e verifica se corresponde. Ao não corresponder, emite uma exceção.

In [ ]:
def verify_Auth(self, message, signature):
        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(message)
        try: 
            h.verify(signature)
            return True
        except cryptography.exceptions.InvalidSignature:
            return False

Relativamente ao decode, utiliza-se o mesmo modo de cifrar, ou seja, AESGCM, com a chave simétrica, o nounce que foi enviado com o mensagem e os metadados. Apenas na posse de todos estes dados é que é possível decifrar o conteúdo enviado.

In [ ]:

def decode (self, ciphertext, nonce, aad):
    aesgcm = AESGCM(self.shared_key)
    try: 
        #decifrar com os mesmos dados da cifra
        plaintext = aesgcm.decrypt(nonce, ciphertext, aad)
    except cryptography.exceptions.InvalidTag:
        return None
    return plaintext.decode('utf-8')


**Assim, criamos uma comunicação assíncrona privada, com a utilização do protocolo DH, utilizando assinaturas DSA, com autenticação do criptograma e metadas usando cifras simétricas num modo HMAC e, ainda, protegende contra ataques a nounces, com geração dos mesmos de forma aleatória.**

* Apresenta-se em baixo um exemplo do resultado dado para a mensagem "Mensagem utilizada para teste":
![](../img/ex1_1.png)



* Outro exemplo para "Outra mensagem para testar":
![](../img/ex1_3.png)